In [1]:
import pandas as pd
import json
import sys
import os
import ast


sys.path.append('../../../')

#import aizynthfinder.chem as chem
import aizynthfinder.analysis

from aizynthfinder.mcts.node import Node
from aizynthfinder.analysis import ReactionTree
from aizynthfinder.mcts.state import State
from aizynthfinder.chem import TreeMolecule
from aizynthfinder.context.collection import ContextCollection
from aizynthfinder.context.stock import StockException

from Maranga.utils.parse_multismiles import findkeys

import matplotlib.pyplot as plt

In [2]:
# read in template file
f = '/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/aizynthfinder/data/uspto_templates.hdf5'
data = pd.read_hdf(f, 'table')

In [12]:
# generate a list of hashs from list of optiised reactiosn
opt_list = ['N-substitution with alkyl-X', 'N-arylation with Ar-X', 'Heteroaryl N-alkylation', 'NH deprotections', 'O-substitution', 'Other functional group interconversion', 'S-substitution', 'O-containing heterocycle formation', 'N-acylation to amide', 'NH protections', 'Other reductions', 'O-acylation to ester', 'Unassigned', 'Stille reaction', 'Halogenation', 'Nitro to amine reduction', 'Reductive amination', 'Dehydration', 'Friedel-Crafts reaction', 'Alkene to alkane', 'Other deprotections', 'Ketone to alcohol', 'ROH deprotections', 'RCO2H deprotections', 'Alcohol to halide', 'Amide to amine reduction', 'O-sulfonylation']

opt_hash = []
for index, row in data.iterrows():
    if row['classification'] in opt_list:
        opt_hash.append(row['template_hash'])


s_opt_hash = list(set(opt_hash))

#save opt list to json 
with open('/data/localhost/not-backed-up/mokaya/exscientia/aizynthfinder/Maranga/experiments/target_2/results/opt4_class/opt_hashes.json', 'w') as f:
    json.dump(s_opt_hash, f)
    

In [23]:
# determine which classes the template refers to:
hash_list = ['9a5276992ccda13b6431b631a7a99deea628d1ae4a1226bec315ced4', '49c1628032788008e8b4aa844c67dfa3dc3da53c87bef32faff5e68f', 'cca685540fa090059d4b84008cff5859b106a1b153e3a038da801bc9', '0d80cfa7f9d1ab63ceb7fdd7712d6120524088f3751de74832ede90b', '69b86575f88b9f0c270424306fd5889c8e4ec122f68301130afe06ca', 'c558acaa3102f970d9cf390a86d77cc5b532eb6c24249bc9fee65f32', '84cba62ead5314a4bd98891d749886ccfe4174585a65261599c12670', '9a4c5ceadb3b4a753bb8f03491f138cd875a84f1d327e818237f90df', '55396c1492cc3823f0701ef923544f9f4c1af584b69f4ce5650dc5ae', '164bcd5926595fb14a6a9bdfa6263711f1e81657ce47f1a9ea439ccb', '7399a60fd7378a24dfa10a10ff432b29a0c80f1c6d7ac70d27cfe818', 'ec4cadfa6747724043a249bd3bb87c9eeb325c6711c85ba00fa25604', 'e9afa703b0ce1ae6beee8caa75610d7a9299fc62a12297fdbddaa49d', '9bf4542b6ebf1b0dbc776e9c158fdb37e9984a15e06ba7fbdad968ad', 'e79b9f0e956cb38254670cbba4f53d0c22148efe9b15465e130440fe', '1eecabe0173fa39f206a365ccea98e2b763ff157e33c23433b777de4', '8f16a628bb96a958262751734b4546192243ce9791e6582406c4ee30', '19ea0b41ce34ad96a8275f097d0c11f47736ca7f973cd7c4aee5b189', 'f4b38824f683bf381c9aa5fd7b484bbde56e2482c8df45c4f3383d61', '4188c1573ceb6063bb6ef444f59b675aa1e5e38b5a55405cdcf8f840', '0700d9a1d582b4bef1511f9fb181aa1e8ef062a6c336f391def96a74', '8143b54e2bc37121732ef6de69f0e94d1a536da302856388446b9673', '44c785fb5a8d587d2ec1ca5060ffda34ecfc782c8160e866fb6e01e9', '858ec45d88e9a7002071b607ca9f50d02f2769cb72fbedc5a4042514', '56d869f2a2ea5e0097f47de129daa2ad7576a8c31ea76ed265f68813', '9700dfbbea18932b10f40ed49527a6efd0b261364d0b5e3b1e240441', 'e21cbc5dc2263b0652396beb92f05fda37bb9f2c606e45a3619c49bf', '6362926c9777bf7356052a388aa6d0f4ebf38f2592d1109c8e9e9c05', 'e2c3ee5da9aa9675ad987be40beb5130d31e54544c07aad9093e3547', 'faafe15908733f62a51dcacbeeefe13083c70721f4a054ab98175e58', '243d4d70ed956e3ba4e5bc0d489e1112d3a0eba27892f1d71f3a13af', 'f50e6edd43215fd09e75ad8277fe209f633bdb24972a58fad4063f8f', '91937bdfb1283fee1b409bb4ff0730f7267518519b1eecb183e669ef', '9291446e8f55b6f86c8d5201b11155cb95dda4254b057ed2d058f721', '21f1516d1bc55470578e44cfd7467daa0d35e9d7afb976a677af3083', '6c83f45f485301792011c63e6835be21ece3be24f5b32a6e45acc5be', 'cd4c8dfb74ae00d4ec8387e8837370d2f9fd4ad5e54ff56c9878f839', '0ef44101f86b26c62fc67b3c5018c4ce5a2b39774c9ad34db6b370ac', '6d638120065ef40c19951cbe4a25b83e2ee66458c9215d54cc3c101c', '99db519e153161d9e1a0b089bc1dd8038bb89ab236fbb46bac2ac4f7', 'f858818c0c1f2f66f02bcbcb6a52f9947e9808c326bd2f9e81f608a4', '96be621effdd90c792899a6a93b780dc3489e8b772511ad36a4015c9', '9cc055322c104538aed37bb35dbba0c3097cb8a350ff59b1bb99eb84', '68ed8bbec90a1ac31b5fbc3e08089ca5738aa5e868a05e35b0069282', '501050fd83a951204010154c86c5c53f5a81532ed38ee6aae2c98ef9', 'b7b6f2efc722a1cbfc7ef551228f4090dbd1a72d44d2cead0bb05872', '3a978d5833c3fb0ea167d02796f59b20773ee25ea8ba4ff2bf389240', '64a6d4910232f1acf64becb85f504144f0b60a2b59776f77c9c8a0ee', '493dfbb0afd083041fcf14557ee7dc8217381b2f2e0ce03b2961712e', '3ce1f4f533aa839698b3882185adcd4524eb6157f8dce3089b2b5a90', 'cb56e889c4d58493f51dcb909ba7fe2cb033e7b9cef243223f406e96', '5f978a847bba3863bf5fb6d96bfd1d3cace2da93615c40c010102dd5', '4d723b71896db992aecbf084cc5d15996027d44894eab13d1628cc69', '5925a99854ae54436c082de1b72e3387e647ee9fcbb0579b101bc40b', 'cd43bf36485e3d947480c17977c7dddb0a0e5816a29f4aeac5d058d3', '89f0035837f1d13d5fcec24e5ca3375cf275a00d3c3a3e14ffedf66e', 'fb44fedb5e2cc8d07bdd0b6fb6f327584934b65e8fb8e97e63bb7532', 'e63c08933c6b733679ff30a8c9ad1998df3c0c8c5978c3a6e4fb6eb2', '34b6dca157bf4378599a4022b359f133254f6e0d7b05225c4a0852ec', '4a8e4deb0b76c422b41da3d3271fd917779c02f14055dcca93e437bf', '9ed3b9447fbafa6505b63e54c66f2bcf14c9f2653d3c0585de45bb14', '0106d81e76bebb85b11172629556291776374f8f143385962b178eb3', '2f6fed0e551b48a2a64b92d204b2d9c396cb66d2fb3210eba31281b5', '485f1a5fe8e94fe10384157e43b0e7c377df1c902531cf76de6081c3', 'f0a489cd79c24ed642d5256650cdb0f98e5344b2c074824b17336ecc', '5c7c98358734164da7cd8fc78e707288537511b909567d4149f86df3', '6421e00bf4ec8a43f511429f89288d00bda92d9e2b0e6b21586b51da', '9376d96863922e887a1f3d93ca23dff993a9d7e5998b24783ac43460', '989f64fd9a06286676d3ada249ae7ab233db3378151ced61e58e0b4d', '397c6727c6e12f6f4cf7284922e95b275029b3d5ae63102208155aee', 'd9439a9c947dce4bab09fd7d3897d86f374c7c7408248952ba7edd29', '60d146fd6d1d5f71f6ef98b9474f4bd0f0f9cec586bef0cc94180dd2', '235b66953c00b522d6e1d24772e28d8d336f3a8b81d2c958e383b55a', '8cbd1a1c9605c696db72cde57e5ef46d3de0246e74379fcf154e8ae7', '7ba139dd17fd258af1de604b040912f547b4175379a289dbd0962986', '9530566a7a51605a818515446d3e2d5bdf0af59e8058923825364b37', '58bfc612ddb3c3923670b0fade9423da56792769e45e6b088debac9e', '226c424c2f0f732effd69df54e67734cf26e4b8b4dc8028c14bdaf6a', '53994dd2d8bb14288cdfb7c2951155e80ca49e2c98f7d531177a38b3', '4b7b1e2a1a99f19df607b2b2df0dc0b449217db0be3b56b070b5dca8', '491647e089969205d82bcdacc23232b9964014fa5e105ea540db7ba8', '874f533cb23b97c467028e6036fbaafdbd21166dcb2c66b52ef718e1', 'c38df6bb72f417e54d2346e687e032ed86554f208af48effa9422912', '95e944b0bcd838bf90fb367a5b7fe79d4dc0134e5def5cf37cca343a', '368b3f927702b41932c1075690d6eeba50254e115711ea09825c0c9c', '0a02d2d47df007d05d29f7b852c5bfe9dcbd7a32224d027cc636bd27', '2198be2381e37a3334f25c59fbeede1694f6fea560c8926b33da47aa', 'e189425e090fce77131c907ebf1468461240dd27d4457a40d701e43e', 'bc7193984167c71913195a0abce257da8748eb7d4b76769dceef2d3f']
templates = {}
for i in hash_list:
    df = data.loc[data['template_hash'] == i]
    if df.iloc[0]['classification'] in templates:
        templates[df.iloc[0]['classification']] += 1
    else:
        templates[df.iloc[0]['classification']] = 1

s_templates = dict(sorted(templates.items(), key=lambda kv: kv[1], reverse=True))
print(s_templates)
print(s_templates.keys())

{'Unassigned': 13, 'Heteroaryl N-alkylation': 12, 'Alcohol to halide': 9, 'Reductive amination': 9, 'NH deprotections': 7, 'N-acylation to amide': 6, 'Other functional group interconversion': 6, 'RCO2H deprotections': 3, 'Halogenation': 2, 'Ketone to alcohol': 2, 'Other reductions': 2, 'ROH deprotections': 2, 'O-containing heterocycle formation': 2, 'N-substitution with alkyl-X': 2, 'N-arylation with Ar-X': 2, 'O-acylation to ester': 2, 'Amide to amine reduction': 1, 'NH protections': 1, 'Stille reaction': 1, 'O-substitution': 1, 'S-substitution': 1, 'Alkene to alkane': 1, 'Friedel-Crafts reaction': 1, 'Nitro to amine reduction': 1}
dict_keys(['Unassigned', 'Heteroaryl N-alkylation', 'Alcohol to halide', 'Reductive amination', 'NH deprotections', 'N-acylation to amide', 'Other functional group interconversion', 'RCO2H deprotections', 'Halogenation', 'Ketone to alcohol', 'Other reductions', 'ROH deprotections', 'O-containing heterocycle formation', 'N-substitution with alkyl-X', 'N-aryl

In [4]:
#convert 'clasification column to list'
classes = data['classification'].tolist()
unique_class = list(set(classes))


In [5]:
for i in unique_class:
    print(i)

Heck reaction
Nitro to amine reduction
Carbamate/carbonate formation
Sulfonation
Ketone to alcohol
Other reductions
Resolution
Unassigned
Other functional group addition
Dehydration
O-sulfonylation
Alcohol to halide
NH protections
Other heteroatom alkylation/arylation
Other functional group interconversion
RSH deprotections
Heteroaryl N-alkylation
Sonogashira reaction
Other organometallic C-C bond formation
Other oxidations
Reductive amination
N-acylation to amide
Stille reaction
Alkene oxidative cleavage
Alkene oxidation
Wittig olefination
Other C-C bond formation
N-containing heterocycle formation
Acid to acid chloride
ROH deprotections
Alcohols to aldehydes
Other acylation
Cyano or imine to amine
Salt formation
N-substitution with alkyl-X
Other deprotections
Oxidations at nitrogen
N-sulfonylation
O-substitution
Alcohols to acids
Alkene to alkane
Other protections
Halogenation
N-arylation with Ar-X
Suzuki coupling
RCO2H protections
RCO2H deprotections
ROH protections
O-containing het

In [4]:
import random

rand19 = random.sample(unique_class, 19)
print(rand19)

rand10 = random.sample(unique_class, 10)
print(rand10)

['NH deprotections', 'O-containing heterocycle formation', 'RCO2H deprotections', 'Other heteroatom alkylation/arylation', 'Acid to acid chloride', 'O-substitution', 'Other reductions', 'Alcohols to aldehydes', 'Grignard reaction', 'Salt formation', 'Amidine formation', 'Sulfonation', 'N-substitution with alkyl-X', 'N-arylation with Ar-X', 'O-acylation to ester', 'Alkene to alkane', 'Other functional group interconversion', 'Friedel-Crafts reaction', 'Other protections']
['O-containing heterocycle formation', 'Nitrile to acid', 'Other functional group interconversion', 'Other Pd-catalyzed reactions (Negishi,Kumada,etc.)', 'RSH deprotections', 'O-acylation to ester', 'N-arylation with Ar-X', 'Alkene to alkane', 'RCO2H deprotections', 'Ester to alcohol']
